In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import spacy
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from fractions import Fraction
import re
from math import log10
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
s1 = "sunshine state enjoy sunshine"
s2 = "brown fox jump high, brown fox run"
s3 = "sunshine state fox run fast"
arr=[s1,s2,s3]
s2=re.sub(r"[,]","",s2)


##-->BOW

In [ ]:
CountV = CountVectorizer(ngram_range=(1,1))
Count = CountV.fit_transform([s1,s2,s3])
BOW_data = Count.toarray()

In [ ]:
BOW_df=pd.DataFrame(BOW_data,columns=CountV.get_feature_names(),index=['s1','s2','s3'])
BOW_df

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
s1,0,1,0,0,0,0,0,1,2
s2,2,0,0,2,1,1,1,0,0
s3,0,0,1,1,0,0,1,1,1


##-->TF-VECTORIZER

In [ ]:
tf=pd.DataFrame(BOW_data,columns=CountV.get_feature_names(),index=['s1','s2','s3'])

tf.loc["s1"] = [Fraction(i,len(s1.split())) for i in tf.loc["s1"]]
tf.loc["s2"] = [Fraction(i,len(s2.split())) for i in tf.loc["s2"]]
tf.loc["s3"] = [Fraction(i,len(s3.split())) for i in tf.loc["s3"]]

In [ ]:
tf

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
s1,0,1/4,0,0,0,0,0,1/4,1/2
s2,2/7,0,0,2/7,1/7,1/7,1/7,0,0
s3,0,0,1/5,1/5,0,0,1/5,1/5,1/5


In [ ]:
#                                                         # WRONG ANSWERS #
# tfVectorizer = TfidfVectorizer(use_idf=False)

# tfData = tfVectorizer.fit_transform([s1, s2, s3])
# tf= np.round_(tfData.toarray(),decimals = 4)

# tf_dataframe=pd.DataFrame(tf,columns=tfVectorizer.get_feature_names())
# tf_dataframe

##-->IDF-VECTORIZER

In [ ]:
features = CountV.get_feature_names_out()
features

array(['brown', 'enjoy', 'fast', 'fox', 'high', 'jump', 'run', 'state',
       'sunshine'], dtype=object)

In [ ]:
total_docs = len(arr)
idf = dict()
for i in range(len(features)):
  v = 0
  if features[i] in s1.split():
    v+=1
  if features[i] in s2.split():
    v+=1
  if features[i] in s3.split():
    v+=1
  idf[features[i]]= log10(Fraction(total_docs,v))

In [ ]:
pd.DataFrame(idf,index=["IDF's"])

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
IDF's,0.477121,0.477121,0.477121,0.176091,0.477121,0.477121,0.176091,0.176091,0.176091


In [ ]:
#                                                        TF-IDF Model(WRONG ANSWERS)
# idfVectorizer = TfidfVectorizer(use_idf=True) 
# idfData = idfVectorizer.fit_transform([s1, s2, s3])
# idf = idfVectorizer.idf_
# # data=idfData.toarray()

# idf_dataframe=pd.DataFrame(idf,columns=['IDF'],index=idfVectorizer.get_feature_names())
# # idf_dataframe

# t1 = np.array(idf[0])
# t3 = np.array(idf[2])
# idf_dataframe
# # # idf = tf.idf_ 

##-->TF-IDF

In [ ]:
#TF-IDF
tf_idf_vec = TfidfVectorizer(use_idf=True) 
tf_idf_data = tf_idf_vec.fit_transform([s1, s2, s3])
tfidf_data=tf_idf_data.toarray()
tf_idf_dataframe=pd.DataFrame(tfidf_data,columns=tf_idf_vec.get_feature_names())
tf_idf_dataframe
t1 = np.array(tfidf_data[0])
t3 = np.array(tfidf_data[2])
tf_idf_dataframe

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
0,0.000000,0.50689,0.000000,0.000000,0.000000,0.000000,0.000000,0.385503,0.771006
1,0.670703,0.00000,0.000000,0.510087,0.335352,0.335352,0.255044,0.000000,0.000000
2,0.000000,0.00000,0.549351,0.417796,0.000000,0.000000,0.417796,0.417796,0.417796


##-->COSINE-SIMILIARITY

In [ ]:
# compute cosine similarity
cosine = np.dot(t1,t3)/(norm(t1)*norm(t3))
print(round(cosine,4))

0.4832
